In [1]:
import pandas as pd
import xlrd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import scipy.stats as sts

In [4]:
#Kinase map table with uniprot ID, Gene ID, String ID
PPI=pd.read_table('PPI/9606.protein.links.v11.5.txt',sep=' ')
uni_to_string_all_kin=pd.read_table('PPI/Uni_to_string_all_kin.txt',sep='\t')
uni_to_id_all_kin=pd.read_table('CoPheeKSA/kinase_network/kinase_uni_to_geneid.tsv')
uni_to_string_all_kin.columns=['Uniprot','string']
uni_to_string_all_kin.index=uni_to_string_all_kin.Uniprot
uni_to_string_all_kin=uni_to_string_all_kin.drop(['Uniprot'],axis=1)
uni_to_id_all_kin.columns=['Uniprot','Gene ID']
uni_to_id_all_kin.index=uni_to_id_all_kin.Uniprot
uni_to_id_all_kin=uni_to_id_all_kin.drop(['Uniprot'],axis=1)
map_table=pd.concat([uni_to_string_all_kin,uni_to_id_all_kin],axis=1)
map_table=map_table.drop_duplicates()

In [13]:
#KSA gold standard
kinases_id=pd.read_excel('KSA/kinase_table.xlsx')
kinases_id.index=kinases_id.Name
K_S_GPS=pd.read_csv('KSA/KSA_gold_standard.csv',index_col=0)

In [30]:
#kinase id
kinase_id=map_table['Gene ID'].tolist()
len(kinase_id)

In [44]:
#ensembl of proteomics to gene id
converter=pd.read_table('PanCan/protein_ids.csv',header=None)
converter=converter.dropna()

#kinase covered in proteomics
kinase_id_inproteomics=list(set(kinase_id).intersection(set(converter[1].tolist())))
len(kinase_id_inproteomics)

390

In [23]:
#data construction
#Protein
#Host protein
BRCA_tumor_pro=pd.read_table('PanCan/Protein/BRCA_proteomics_gene_abundance_log2_reference_intensity_normalized_Tumor.cct',index_col='idx')
CCRCC_tumor_pro=pd.read_table('PanCan/Protein/CCRCC_proteomics_gene_abundance_log2_reference_intensity_normalized_Tumor.cct',index_col='idx')
COAD_tumor_pro=pd.read_table('PanCan/Protein/COAD_proteomics_gene_abundance_log2_reference_intensity_normalized_Tumor.cct',index_col='idx')
GBM_tumor_pro=pd.read_table('PanCan/Protein/GBM_proteomics_gene_abundance_log2_reference_intensity_normalized_Tumor.cct',index_col='idx')
HCC_tumor_pro=pd.read_table('PanCan/Protein/HCC_proteomics_gene_abundance_log2_reference_intensity_normalized_Tumor.cct',index_col='idx')
HNSCC_tumor_pro=pd.read_table('PanCan/Protein/HNSCC_proteomics_gene_abundance_log2_reference_intensity_normalized_Tumor.cct',index_col='idx')
LSCC_tumor_pro=pd.read_table('PanCan/Protein/LSCC_proteomics_gene_abundance_log2_reference_intensity_normalized_Tumor.cct',index_col='idx')
LUAD_tumor_pro=pd.read_table('PanCan/Protein/LUAD_proteomics_gene_abundance_log2_reference_intensity_normalized_Tumor.cct',index_col='idx')
OV_tumor_pro=pd.read_table('PanCan/Protein/OV_proteomics_gene_abundance_log2_reference_intensity_normalized_Tumor.cct',index_col='idx')
PDAC_tumor_pro=pd.read_table('PanCan/Protein/PDAC_proteomics_gene_abundance_log2_reference_intensity_normalized_Tumor.cct',index_col='idx')
UCEC_tumor_pro=pd.read_table('PanCan/Protein/UCEC_proteomics_gene_abundance_log2_reference_intensity_normalized_Tumor.cct',index_col='idx')

In [24]:
tmp_new_pro=[]
tmp=[BRCA_tumor_pro,CCRCC_tumor_pro,COAD_tumor_pro,GBM_tumor_pro,HCC_tumor_pro,HNSCC_tumor_pro,
     LSCC_tumor_pro,LUAD_tumor_pro,OV_tumor_pro,PDAC_tumor_pro,UCEC_tumor_pro]
for d in range(len(tmp)):
    DATA=tmp[d]
    ks=[]
    DATA.replace([np.inf, -np.inf], np.nan, inplace=True)
    idx_80=[]
    for k in range(len(DATA)):
        if sum(DATA.iloc[k,:].isna())/len(DATA.iloc[0,:])<=0.8:
            idx_80.append(k)
    DATA=DATA.iloc[idx_80,:]
    tmp_new_pro.append(DATA)

In [49]:
tmp_new_pro_id=[]
for i in range(len(tmp_new_pro)):
    data=tmp_new_pro[i]
    data['gene_id']=np.repeat('nan',len(data))
    idx=[]
    for k in range(len(data)):
        if data.index[k] in converter[0].tolist():
            tmp=converter[converter[0]==data.index[k]]
            idx.append(k)
            name=tmp[1].iloc[0]
            data['gene_id'].iloc[k]=name
    data=data.iloc[idx]
    tmp_new_pro_id.append(data)

/var/folders/2k/p13z68bx7p9fvh4xn3mbjn_r0000gr/T/ipykernel_39875/3557172456.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['gene_id']=np.repeat('nan',len(data))
/Users/wj6/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [51]:
#PCC between kinases
k_s_features={}
for i in range(len(kinase_id_inproteomics)-1):
    for j in range(i+1,len(kinase_id_inproteomics)):
        kinase1=kinase_id_inproteomics[i]
        kinase2=kinase_id_inproteomics[j]
        PCC_K=[]
        for k in range(len(tmp_new_pro_id)):
            tmp1=tmp_new_pro_id[k]
            abun1=tmp1[tmp1.gene_id==kinase1]
            abun2=tmp1[tmp1.gene_id==kinase2]
            if (len(abun1)>0) and (len(abun2)>0):
                abun1=abun1.iloc[0,:-1]
                abun2=abun2.iloc[0,:-1]
                try:
                    corr = sts.spearmanr(abun1, abun2,nan_policy='omit')
                    PCC_K.append(corr[0])
                except:
                    PCC_K.append(np.nan)
            else:
                PCC_K.append(np.nan)
        idx='*'.join([kinase1,kinase2])
        k_s_features[idx]=np.array(PCC_K)

In [52]:
k_s_features=pd.DataFrame(k_s_features)
k_s_features=k_s_features.transpose()
k_s_features['max_pcc']=k_s_features.max(axis=1)
k_s_features_filtered=k_s_features[k_s_features.max_pcc>=0.5]

In [54]:
Cancer=['BRCA','CCRCC','COAD','GBM','HCC','HNSCC','LSCC','LUAD','OV','PDAC','UCEC','max_PCC']

In [55]:
k_s_features.columns=Cancer

In [56]:
k_s_features.to_csv('CoPheeKSA/kinase_network/PCC_KK.csv')

In [57]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [58]:
PPI_400=PPI[PPI['combined_score']>=400]

In [ ]:
#PPI
k_s_features_PPI={}
pos_PPI_score=[]
idx=[]
for i in range(len(map_table)-1):
    print(i)
    for j in range(i+1,len(map_table)):
        kinase1=map_table['Gene ID'].iloc[i]
        kinase2=map_table['Gene ID'].iloc[j]   
        string1=map_table['string'].iloc[i]
        string2=map_table['string'].iloc[j]

        tmp=PPI_400[PPI_400['protein1']==string1]
        tmp=tmp[tmp['protein2']==string2]
        if len(tmp)>0:
            pos_PPI_score.append(tmp.combined_score.iloc[0])
            idx.append('*'.join([kinase1,kinase2]))

In [60]:
k_s_features_PPI['PPI']=np.array(pos_PPI_score)
k_s_features_PPI=pd.DataFrame(k_s_features_PPI)
k_s_features_PPI.index=idx

In [61]:
k_s_features_PPI.to_csv('CoPheeKSA/kinase_network/PPI_KK.csv')

In [74]:
KK_pos1=k_s_features_PPI[k_s_features_PPI['PPI']!=0]
KK_pos2=k_s_features[k_s_features.max_PCC>=0.5]
KK_pos=pd.concat([KK_pos1,KK_pos2])

In [75]:
KK_pos=KK_pos.drop_duplicates()

In [76]:
lst1=[i.split('*')[0] for i in KK_pos.index]
lst2=[i.split('*')[1] for i in KK_pos.index]

In [77]:
len(set(lst1+lst2))

352

In [81]:
KK={}
KK['node1']=np.array(lst1)
KK['node2']=np.array(lst2)
KK=pd.DataFrame(KK)

In [83]:
KK.to_csv('CoPheeKSA/kinase_network/KK_PPI_PCC.csv')

In [105]:
KK=pd.read_csv('CoPheeKSA/kinase_network/KK_PPI_PCC.csv',index_col=0)

In [106]:
KK=KK.drop_duplicates()

In [107]:
len(set(KK.node1.tolist()+KK.node2.tolist()))

352

In [108]:
len(KK)

3199

In [109]:
import csrgraph as cg
import nodevectors
from node2vec import Node2Vec as n2v
from scipy.linalg import fractional_matrix_power
import networkx as nx

In [110]:
ks1=KK.node1.tolist()
ks2=KK.node2.tolist()

In [ ]:
G = nx.Graph()
elist=list(zip(ks1,ks2))
G.add_edges_from(elist)

In [111]:
#Node2Vec
WINDOW = 1 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 4 # Node2Vec batch words

g_emb = n2v(
  G,
  dimensions=16
)

mdl = g_emb.fit(
    vector_size = 16,
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS
)

Computing transition probabilities:   0%|          | 0/352 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|████████████████| 10/10 [00:01<00:00,  6.68it/s]


In [112]:
emb_df = (
    pd.DataFrame(
        [mdl.wv.get_vector(str(n)) for n in G.nodes()],
        index = G.nodes
    )
)

In [113]:
emb_df.to_csv('CoPheeKSA/kinase_network/n2v_KK_PPI_PCC.csv')

In [114]:
emb_df=pd.read_csv('CoPheeKSA/kinase_network/n2v_KK_PPI_PCC.csv',index_col=0)

In [115]:
emb_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
AKT1,-0.607924,0.610055,-0.285088,-0.203638,0.859098,0.408110,0.999433,0.032701,0.544129,-0.194365,0.016191,0.491396,-0.586680,-0.692457,0.037612,0.507755
AKT2,-0.506512,0.761550,0.885375,0.552965,1.694904,0.108566,0.673667,-0.108241,-0.105654,1.087202,-0.915874,-0.185796,-0.512442,-0.787983,0.286040,1.132861
AKT3,0.313800,0.537653,0.459754,0.400461,0.360645,0.202075,0.899717,0.688519,0.399713,0.552113,0.116394,0.090723,0.133793,-1.693714,0.236023,0.386526
PDPK1,-0.431848,0.026374,0.628174,0.550229,0.022869,-0.647270,1.265898,0.535040,0.377171,0.161587,-0.743762,-1.086578,0.109169,-0.944147,0.527221,0.641994
PRKCB,-0.184234,-0.537358,0.570782,0.843878,0.227461,-0.016780,0.973617,0.450593,0.142778,0.069527,-0.710573,0.069166,-0.093114,-0.814852,-0.138265,0.017476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MERTK,0.586165,-0.340174,0.558953,-0.325136,1.282936,0.361652,0.639148,0.573874,0.013084,0.290880,-1.361647,0.298338,-0.238157,0.182921,0.025928,0.126546
WNK3,0.490734,0.885619,-0.451342,-0.703977,0.969128,1.047832,0.975775,0.804948,-0.243400,0.170834,-0.662291,-1.557295,0.697016,-0.538556,0.071816,-0.736783
ADCK5,1.333178,0.261898,0.487935,-1.168263,1.202449,-0.981057,2.320613,0.600104,-0.259838,0.775453,-0.517834,-0.309033,0.419547,-0.434701,-0.631191,0.772554
TBCK,0.431163,-0.616645,0.309328,0.642366,0.334944,-0.108466,0.816451,0.102297,0.014902,0.020228,-0.144082,-0.232678,0.930311,-1.150585,0.151183,0.030719


In [142]:
#CoPheeMap
network=pd.read_csv('CoPheeMap/data_construction/model_all_pos/All_positive_thred_9679.csv',index_col=0)
seq1=[i.split('|')[3] for i in network.index.tolist()]
seq2=[i.split('|')[3] for i in network.site2.tolist()]
network['seq1']=np.array(seq1)
network['seq2']=np.array(seq2)
sites=list(set(network.index.tolist()+network.site2.tolist()))
G = nx.Graph()
elist=list(zip(network.index.tolist(),network.site2.tolist()))
G.add_edges_from(elist)

idx_ST=[]
idx_Y=[]
for i in range(len(network)):
    if network.seq1.iloc[i][7] in ['S','T']:
        idx_ST.append(i)
    if network.seq1.iloc[i][7] in ['Y']:
        idx_Y.append(i)
network_ST=network.iloc[idx_ST,:]
network_Y=network.iloc[idx_Y,:]
lst_ST=list(set(network_ST.index.tolist()+network_ST.site2.tolist()))
H=G.subgraph(lst_ST)
nodes_ST=max(nx.connected_components(H))
G_ST=G.subgraph(nodes_ST)
network_Y=network.iloc[idx_Y,:]
lst_Y=list(set(network_Y.index.tolist()+network_Y.site2.tolist()))
H=G.subgraph(lst_Y)
nodes_Y=max(nx.connected_components(H))
G_Y=G.subgraph(nodes_Y)

In [143]:
#Node2Vec
WINDOW = 1 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 4 # Node2Vec batch words

g_emb = n2v(
  G_ST,
  dimensions=16
)

mdl = g_emb.fit(
    vector_size = 16,
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS
)

Computing transition probabilities:   0%|          | 0/25028 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|████████████████| 10/10 [10:56<00:00, 65.63s/it]


In [145]:
emb_df = (
    pd.DataFrame(
        [mdl.wv.get_vector(str(n)) for n in G_ST.nodes()],
        index = G_ST.nodes
    )
)

In [146]:
emb_df.to_csv('CoPheeMap/data_construction/model_all_pos/n2v_networkST.csv')

In [147]:
emb_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
ENSG00000171241.9|ENSP00000306473.3|S273|SSLSNCNSDSEQENI|1,0.533069,0.269978,-0.397989,1.340892,1.335203,2.225864,-0.175958,-0.724254,0.516803,0.826250,-0.643912,-1.253300,-0.149629,-2.406677,1.666364,0.557810
ENSG00000092853.14|ENSP00000312995.3|S67|KVLQDSDSETEDTNA|1,0.103202,0.191396,-0.394215,0.698219,0.473290,1.130238,-0.039987,-0.581319,0.306546,0.121685,-0.901712,-0.701528,0.283831,-1.233030,-0.071715,0.424047
ENSG00000109805.10|ENSP00000251496.2|T931|KNKEVYMTPLRGVKA|1,1.674164,-1.382469,0.091658,0.030891,1.898655,1.990421,-0.089816,-1.073188,0.325161,0.887386,-1.511659,-0.770961,0.023637,-2.628052,-1.312919,1.726822
ENSG00000080986.13|ENSP00000261597.4|T31|VNKQGLYTPQTKEKP|1,0.303931,-0.277460,0.149636,0.470600,0.864672,0.719516,0.054380,-0.300457,0.419328,0.069352,-1.130390,0.324361,0.078634,-0.865294,-0.371673,0.400288
ENSG00000085840.13|ENSP00000360623.3|S199|AKSKSAESPSWTPAE|1,0.835798,-0.210644,-0.304174,0.836571,1.055412,2.183620,-1.004867,-0.104965,-0.396849,0.052140,-1.409207,-1.384124,0.090759,-1.751354,0.723015,1.731726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000101940.18|ENSP00000365919.5|S111|GARGHRRSVSRGSYQ|1,-0.820531,-0.311317,-0.125550,1.018275,-0.121492,-0.098329,-0.410019,-0.013194,0.140103,0.276340,-0.938218,0.673987,-0.094440,-0.865158,-0.392984,0.703430
ENSG00000133401.16|ENSP00000402033.1|S1309|YSKTRSASETSTPHN|1,-1.024809,-0.265445,0.395480,0.330470,0.228199,0.409054,-0.572364,-0.375212,-1.063648,1.409686,-1.061344,0.323769,-0.917333,-0.113788,-0.496233,0.806499
ENSG00000134982.17|ENSP00000257430.4|S2207|TGKVRSNSEISGQMK|1,0.042654,-0.380979,-0.372519,1.214377,-0.351453,0.401634,-0.414095,-0.147979,-0.616606,0.734896,-0.456842,0.577219,-0.177840,-0.737484,-0.154334,0.008302
ENSG00000153827.13|ENSP00000283943.4|S161|SSCVKSGSGSESTGA|1,0.117526,-0.970980,0.498546,0.562464,0.833776,-0.457409,-0.569688,-1.486665,-0.177795,0.312324,0.632640,0.026241,-0.562320,-0.268987,0.369797,0.841009
